# Setup

In [1]:
%matplotlib ipympl
import matplotlib, matplotlib.pyplot

matplotlib.pyplot.ioff()

from package.apps import Apps

# Queries

## Apps using IValidator

In [8]:
with Apps.init() as apps:
    apps.select(
        {
            "IValidators": lambda app: app.program_cs.custom_app_services(interface_name="IValidator").map(
                lambda class_name: app.cs.flat_map(
                    lambda cs: cs.class_declarations(name=class_name, implements="IValidator")
                ).first
            )
        }
    ).where(lambda app: len(app["IValidators"]) > 0).table()

Env Org App IValidators 
 
 
 tt02 brg aarsregnskap-bank-202404 aarsregnskap-bank-202404/App/logic/Validation/TaskValidator.cs 18 public class TaskValidator : IValidator 
 19 { 
 20 private IAppResources _appResourceHandler ; 
 21 private static TextResource _textResource ; 
 22 private readonly IHttpContextAccessor _httpContextAccessor ; 
 23 private readonly IDataClient _dataClient ; 
 24 private string instanceId ; 
 25 private int partyGuid ; 
 26 private Guid instanceGuid ; 
 27 private string org ; 
 28 private string app ; 
 29 private List < DataElement > dataModels ; 
 30 private DataElement hovedskjema ; 
 31 private string hovedskjemaId ; 
 32 private Type dataType ; 
 33 private ModelStateDictionary _validationResults ; 
 34 private DateTime startdato ; 
 35 
 36 public TaskValidator ( IDataClient dataClient ) 
 37 { 
 38 _dataClient = dataClient ; 
 39 } 
 40 private Task < object > GetForms ( Instance instance ) 
 41 { 
 42 instanceId = instance . Id . Contains ( '/' ) ? instance . Id . Split ( "/" )[ 1 ] : instance . Id ; 
 43 partyGuid = int . Parse ( instance . InstanceOwner . PartyId ); 
 44 instanceGuid = new Guid ( instanceId ); 
 45 org = instance . Org ; 
 46 app = instance . AppId ; 
 47 dataModels = instance . Data ; 
 48 //bruk datatype til hovedskjema 
 49 hovedskjema = dataModels . Find ( o => o . DataType == "Hovedskjema" ); 
 50 hovedskjemaId = hovedskjema . Id ; 
 51 //spesifiser underskjema type 
 52 dataType = typeof ( Altinn . App . Models . RR0003H_M . RR0003H_M ); 
 53 var task = _dataClient . GetFormData ( instanceGuid , dataType , org , app , partyGuid , new Guid ( hovedskjemaId )); 
 54 
 55 return task ; 
 56 } 
 57 
 58 public async Task < List < ValidationIssue >> Validate ( IInstanceDataAccessor dataAccessor , string taskId , string language ) 
 59 { 
 60 if ( ! taskId . Equals ( "Utfylling" )) return null ; 
 61 
 62 var datamodelHovedskjema = GetForms ( dataAccessor . Instance ); 
 63 var datamodelHovedskjemaObjects = await Task . WhenAll ( datamodelHovedskjema ); 
 64 var modelHovedskjema = datamodelHovedskjemaObjects . Cast < RR0003H_M > (). Single (); 
 65 
 66 // Sjekk på krav om å legge ved "vedlegg for bærekraftsrapportering" 
 67 var taksonomi = modelHovedskjema . Skjemainnhold ?. aarsberetning ?. taksonomi ; 
 68 var startDato = modelHovedskjema . Skjemainnhold ?. regnskapsperiode ?. regnskapsstart ?. value ; 
 69 
 70 DateTime . TryParse ( startDato , out startdato ); 
 71 
 72 var baerekraftVedlegg = dataAccessor . Instance . Data . Where ( i => i . DataType == "VedleggBaerekraft" ); 
 73 var harBaerekraftVedlegg = baerekraftVedlegg . Any (); 
 74 
 75 List < ValidationIssue > validationIssues = new (); 
 76 if (( taksonomi != null && taksonomi . Count > 0 ) && ( startdato . Year >= 2024 ) && ( harBaerekraftVedlegg == false )) 
 77 { 
 78 validationIssues . Add ( new ValidationIssue 
 79 { 
 80 DataElementId = "VedleggBaerekraft" , 
 81 Description = "An attachment (sustainability reporting) is required." , 
 82 CustomTextKey = "VedleggBaerekraft.message" , 
 83 Severity = ValidationIssueSeverity . Error 
 84 }); 
 85 } 
 86 return validationIssues ; 
 87 } 
 88 
 89 public Task < bool > HasRelevantChanges ( IInstanceDataAccessor dataAccessor , string taskId , DataElementChanges changes ) 
 90 { 
 91 //throw new System.NotImplementedException(); 
 92 return Task . FromResult ( true ); 
 93 } 
 94 
 95 public string TaskId { get ; } = "Utfylling" ; 
 96 } 
 
 
 tt02 brg aarsregnskap-funk-202405 aarsregnskap-funk-202405/App/logic/Validation/TaskValidator.cs 18 public class TaskValidator : IValidator 
 19 { 
 20 private IAppResources _appResourceHandler ; 
 21 private static TextResource _textResource ; 
 22 private readonly IHttpContextAccessor _httpContextAccessor ; 
 23 private readonly IDataClient _dataClient ; 
 24 private string instanceId ; 
 25 private int partyGuid ; 
 26 private Guid instanceGuid ; 
 27 private string org ; 
 28 private string app ; 
 29 private List

Count: 8


## Apps in prod using IFormDataValidator

In [5]:
with Apps.init() as apps:
    apps.select(
        {"IFormDataValidators": lambda app: app.program_cs.custom_app_services(interface_name="IFormDataValidator")}
    ).where(lambda app: len(app["IFormDataValidators"]) > 0).table()

Env,Org,App,IFormDataValidators
prod,digdir,be-om-api-nokkel,[FormDataValidator]
prod,digdir,bli-tjenesteeier,[FormDataValidator]
prod,digdir,soknad-om-stimulabmidler,[FormDataValidator]
tt02,brg,aarsregnskap-bank-202404,"[ValidationHandler, ValidationHandlerUnderskjema]"
tt02,brg,aarsregnskap-begr-202405,"[ValidationHandler, ValidationHandlerUnderskjema]"
tt02,brg,aarsregnskap-friv-202402,[ValidationHandler]
tt02,brg,aarsregnskap-funk-202405,"[ValidationHandler, ValidationHandlerUnderskjema]"
tt02,brg,aarsregnskap-liv-202404,"[ValidationHandler, ValidationHandlerUnderskjema]"
tt02,brg,aarsregnskap-pensjon-202404,"[ValidationHandler, ValidationHandlerUnderskjema]"
tt02,brg,aarsregnskap-skade-202404,"[ValidationHandler, ValidationHandlerUnderskjema]"


Count: 302
